In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib.cm as cmap
import os

# Download the custom library - do this manually if not running on google colab
if not os.path.isfile("RL_library.py"):
    try:
        from pydrive.auth import GoogleAuth
        from pydrive.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
    except ImportError:
        raise ImportError("Please download RL_library.py manually from the github")
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    _f = drive.CreateFile({'id': "1Mn9GLnUkgCc2deBna2X9-uAy_wjkuX-g"})
    _f.GetContentFile('RL_library.py')

# custom package
from RL_library import *

## RL simulations

This code is related to:

Bob Wilson & Anne Collins (2019) eLife: Ten simple rules for the computational modeling of behavioral data.

In this paper, the authors describe a 2-armed bandit, where a player performs $T$ choices between two options. The machine has asymmetric reward probabilities $\mu_{1} = 0.2$ and $\mu_{2} = 0.8$ associated with each arm, which are initially unknown to the player.

We want to find out: Which strategy does a subject use to maximize their overall reward on this machine?

## 0. Helper functions

## 1. Define our experiment parameters to simulate data from each model.

In [3]:
# set numpy seed to 0


In [4]:
# experiment parameters
T   = 100                    # number of trials
mu  = np.array([0.2, 0.8])   # mean reward of each bandit

Nrep = 110 # number of repetitions of the simulation

## 2. Simulate our candidate models.

We will consider three different normative models (M2, M3, and M4) of how subjects might attack the problem of maximizing reward on a 2-armed bandit. You can read the mathematical formulation of each model in the paper. Shortly, the proposed models are:

- **Noisy win-stay-lose-shift**: Rewarded actions are repeated, unrewarded actions lead to the player choosing a different arm in the next trial. Every now and then, the subject explores a different option (model 2 with 1 parameter, $\epsilon$, describing the overall level of randomness)

- **Rescorla Wagner**: In each trial, subjects update the expected values of each option (Q), based on the history of previous outcome. Then, they use these values to make the next decision. Every now and then, they explore the low-value option (model 3 with 2 parameters: learning rate $\alpha_{RW}$, softmax inverse temperature $\beta_{RW}$)

- **Choice kernel**: Subjects have a tendency to repeat responses, the strength of which depends on the history of actions they took before the current trial, which is tracked in the "choice kernel" (model 4 with 2 parameters, choice-kernel learning rate $\alpha_{CK}$, choice-kernel inverse temperature $\beta_{CK}$)

### 2.1. Win-stay-lose-shift

Simulate Model 2, the win-stay-lose-shift (M2WSLS) by calling `simulate_M2WSLS_v1` with $\epsilon=0.1$, which is coded up in `RL_library.py`. 

This simulation process (as well as all the others in part 2. of the tutorial) should be repeated several times (`Nrep`), with different parameter settings, to get a handle on how the model behaves as a function of its parameters. 

In [4]:
# Model 2: Win-stay-lose-shift



### 2.2. Rescorla Wagner

While all models are coded up in `RL_library.py`, try out yourself to write a function for simulating Rescorla Wagner simulate_M3RescorlaWagner_v1 for Model 3 (code instructions below). If you are stuck, have a look at the implementation in the library.

Then, simulate this model with $\alpha_{RW} = 0.1$ and $\beta_{RW} = 5$ and collect resulting actions and rewards. 

In [5]:
# def simulate_M3RescorlaWagner_v1(T, mu, alpha, beta):
# learning rate alpha, softmax parameter beta

    # initialise Q values for each of the two options: initially, no preference

    # loop over trials

        # compute choice probabilities using softmax formula

        # save actions and rewards
        
        # update values using prediction error term

#     return a, r # return sequence of actions and rewards

In [6]:
# Model 3: Rescorla Wagner


### 2.3. Choice kernel

Finally, simulate a choice kernel model by calling `simulate_M4ChoiceKernel_v1` with $\alpha_{CK}=0.1$ and $\beta_{CK}=3$ (model 4).

In [7]:
# Model 4: Choice kernel


In [8]:
# Model 5: Rescorla-Wagner + choice kernel


For each model, you should also play with the parameters used to generate the simulations and observe the effect on Win-Stay-Lose-Shift analysis (see below). 

## 3. Behavioral analyses: Visualise the behaviour of different simulated models.				

We will now observe the behavioral outcomes resulting from each simulation. A typical analysis that might allow you to compare qualitatively the model behavior with actual subjects playing the 2-armed bandit is a win-stay-lose-shift analysis that plots the probability of repeating an action, p(stay), as a function of the previous reward (see Wilson & Collins, box 2 figure 1).

### 3.1. Compare qualitative patterns from our three different models

Collect win-stay-lose-shift behavior for all three models. Write a function that calculates $p(stay_{n} | win_{n-1} = 0)$ and $p(stay_{n} | win_{n-1} = 1)$ from a single sequence of simulated actions and rewards.

Then, average probabilities across simulations for each model.

In [9]:
# def wsls(a, r):

#     return loseStay, winStay

In [10]:
# Loop over your models

names = ['WSLS', 'Rescorla-Wagner', 'Choice kernel']

# calculate WSLS with shape (3 x 2), # n_models x n_prev_outcomes


Now plot WSLS behavior as a function of previous reward (1 for rewarded, 0 for unrewarded).

- Does each model modulate its probability of staying as a function of previous reward? 

In [5]:
# Plot WSLS as a function of previous reward (1 for rewarded, 0 for unrewarded).


You should see that the choice kernel model leads to a reward-independent $p(stay)$, because choice probabilities are calculated independently of the previous reward. All other models show outcome-modulated behavior, with the starkest differences for the WSLS simulation.

*Take home message*: More broadly, these patterns of behavior can then be contrasted again actual behavioral data to inform about subjects' behavior. It is important to simulate your candidate models and plot their behavior before comparing them to actual data. 

### 3.3. Let's analyse the performance of Model 3: p(correct) analysis

For Rescorla Wagner, we are now interested in how learning rate and softmax inverse temperate affect the probability of choosing the arm with the highest reward. 

We will repeatedly perform a grid search over different parameter values (~ 1000 simulations with 100 trials per grid point) and store the mean $p(correct)$ across trials for each grid point and repetition.

In [6]:
# % Define a range of parameter values to loop over for each parameter.
alphas = np.linspace(0.02, 1, 20)
betas  = np.array([1, 2, 5, 10, 20])

In [7]:
# Let's first use only 10 simulations for each parameter combination. When your code works, increase to 1000.
Nreps = 10

# Initialization

# Grid-search over alpha and beta parameters for a large number of simulations
# on which you will then average.

    # simulate Model 3 and collect sequence of actions and rewards

    # collect the performance

    # store performance for early (first 10 trials) and late (last 10 trials of a block) trials

Now plot $p(correct)$ as a function of $\alpha$ and $\beta$. Create the figure with two subplots: one for early, one for late trials.

As in Wilson & Collins box 2 figure 1, plot different levels of $\alpha$ on the x-axis and use different curves for $\beta$ levels.

In [8]:
# Your figure with two subplots here

How does performance change as a function of alpha and beta parameter values, for early and late trials?

The left graph shows that the learning rate is positively correlated with increases in early performance only for low $\beta$ values, or very low $\alpha$ values. For high $\beta$ values, there is a U-shaped relationship between learning rate and early speed of learning. The right graph shows that with high $\beta$ values, high learning rates negatively influence asymptotic behavior. Thus, both parameters interact to influence both the speed of learning and asymptotic performance. 

*Conclusion*: This kind of analysis will allow you to see qualitative differences between models, so that making their predictions in the experimental setup different. If the behavior of different models is not qualitatively different, this is a sign that you should try to design a better experiment. While not always possible, distinguishing between models on the basis of qualitative patterns in the data is preferable to quantitative model comparison. 

## 4. Parameter recovery

### 4.1. Simulation and fitting

Simulate Model 3 to generate synthetic data, then fit these data.

We will do this step by step. We will first simulate actions of the model for a given learning rate, $\alpha$, and softmax parameter, $\beta$.			

After simulating the model, we will fit the parameters using a maximum likelihood approach to get estimated values $\hat{\alpha}$ and $\hat{\beta}$. 

In [14]:
# experiment parameters


Fit the simulated data. 

Fit Rescorla Wagner by calling `fit_M3RescorlaWagner_v1`. As an input, enter simulated actions and rewards. As an output, extract the fitted parameter values, the loglikelihood LL and the BIC.

We will not focus on the goodness of fit here, but you should take a moment to look at the specification in `fit_M3RescorlaWagner_v1` and `lik_M3RescorlaWagner_v1`.

In [9]:
# sample random parameters
     # learning rate from a uniform distribution between 0 and 1
     # softmax parameter from exponential with mean = 10

# loop over repetitions 

    # set different fixed seeds per repetition
    
    # simulate M3

    # fit and store parameters


### 4.1. Parameter recovery plots

We would like to visualise the fitted parameter values as a function of the generating parameter values. We will have one data point for each iteration. Here, you should use the generating values and the fitted values that you have stored in the section above. You should create two subplots, one for each parameter.

In [10]:
# Your figure with two subplots goes here


Do you observe a fairly good agreement between the simulated and fit parameter values?

The plot makes any correlations clear, and also reveals whether the correlation holds in some parameter regimes but not others. It also reveals any existing bias (e.g. a tendency to recover higher or lower values in average).

Here we can see that the fit for $\beta$ is best with a range, $0.1 < \beta < 10$ and that outside this range, the correspondence between simulation and fit is not as good.

Depending on the values of $\beta$ that we obtain when fitting human behavior, this worse correspondence at small and large $\beta$ may or may not be problematic. It may be a good idea to use the range of parameters obtained from fitting the real data to test the quality of recovery within the range that matters. 

Reliable parameter recovery is particularly important for look at inter-individual differences in relation to questionnaire scores or brain data.

## 5. Model recovery: confusion matrix

To illustrate model recovery, here we will simulate behavior of our three models on the two-armed bandits task. 

As before, the means $\mu$ can be set at 0.2 and 0.8, and the number of trials at $T = 1000$. For each simulation, model parameters can be sampled randomly for each model. 

Each simulated data set will then be fit to each of the given models, to determine which model fit best (according to BIC). This process will be repeated 100 times (number of "repetitions" or "counts") to compute the confusion matrix.

In [11]:
# hyperparameters mu and T
mu = np.array([.2, .8])
T  = 1000

Let's build a confusion matrix. It quantifies the probability that each model is the best fit to data generated from the other models. In a perfect world the confusion matrix will be the identity matrix, but in practice this is not always the case.

How to read the confusion matrix? Given a winning model (a particular column), it tells you the likelihood of each ground-truth model (each row) to have generated the data (basically, the columns are more important than the rows).

In [12]:
# make list of fitting functions to loop through
funs = [fit_M2WSLS_v1, fit_M3RescorlaWagner_v1, fit_M4ChoiceKernel_v1]

# initialise your confusion matrix: 3 by 3 (for our three models).

# Let's loop over number of repetitions: start with 10, increase to 100 if everything works

    # set different fixed seed for each repetition

    #  simulate Model 2 with random epsilon parameter (sampled from uniform [0,1])
    # fit models

    #  simulate Model 3 with random alpha and beta parameters
    # initialize from uniform [0,1] and 1+exponential with mean 1
    # fit models
    
    #  simulate Model 4 with random alpha and beta parameters
    # initialize from uniform [0,1] and 1+ exponential with mean 1
    # fit models
    


In [13]:
# normalize confusion matrix


In [14]:
# plot and print values


- Does your confusion matrix have large off-diagonal components? If so, this indicates that you have a problem with model recovery.

- If you change the generating parameters, in particular the softmax temperature, do you get a better confusion matrix?

You can observe then that the confusion matrix can be so dependent on the simulating parameter values means that it is crucial to match the simulation parameters to the actual fit parameters of your actual behavioral data (when you have some) as best as possible. Models that are identifiable in one parameter regime may be impossible to distinguish in another regime.

A final note to remember: As with all model comparisons, it only tells you which model fits best of the models you considered. In and of itself, this is rather limited information as there are infinitely many other models that you did not consider. This makes it imperative to start with a good set of models initially, that rigorously capture the competing hypotheses.